<a href="https://colab.research.google.com/github/chi-yan/notebooks/blob/master/NLP_(Bible_Text_Encoding_with_the_Universal_Text_Encoder)%2C_similarity_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Google Colab notebook uses Natural Language Processing (NLP) techniques 
to detect Bible verses that are most similar to a random block of text. Word numerical (high-ordered vectors) obtained via the "Universal Text Encoder" is used to numerically model these verses.`

Cosine similarity is to quantify similarity (in meaning) between sentences. A similiarity close to 0 indicates that two sentences are similar in meaning.

References:

http://www.ijstr.org/final-print/dec2019/Lexical-And-Semantic-Analysis-Of-Sacred-Texts-Using-Machine-Learning-And-Natural-Language-Processing.pdf (see Section 3.4)

https://tfhub.dev/google/universal-sentence-encoder/1

In [40]:
import numpy as np # linear algebra
import pandas as pd #
import tensorflow_hub as hub
import csv
import pprint
from tabulate import tabulate
from collections import OrderedDict
from operator import itemgetter
text = 'cats and dogs' #@param {type:"string", run:"auto"}
 
def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''
    # you have to set this variable to the true label.
    cos = -10
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)
 
    return cos
 
if 'init' not in locals():
  init = True
  !curl https://raw.githubusercontent.com/EswarGitHub/BibleSearch/master/bible_data_set.csv -o bible_data_set.csv
  reader = csv.reader(open('bible_data_set.csv', 'r'))
  d = {}
  for row in reader:
    k,_,_,_,v = row
    d[k] = v[:-2] #    
  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  embeddings = embed(list(d.values()))
  versedict = {}
  for i, verse in enumerate(d):
    versedict[i] = verse
 

In [41]:

def cosine_similarities(A, B): #A is the big matrix containing all embeddings, B is the sentence
    return np.divide(np.dot(A,B),np.apply_along_axis(np.linalg.norm, 1, A)*np.linalg.norm(B))[:,0]

def generateSimilarities(text, numberOfItems):
  embedded_text = embed([text])
  similarities = cosine_similarities(embeddings.numpy(), embedded_text.numpy().T)
  data = []
  for i in range(len(embeddings)):
    data.append([versedict[i],d[versedict[i]],similarities[i]])
  return(pd.DataFrame (data, columns = ['Verse', 'Text', 'Similarity']).sort_values(by=['Similarity'], ascending=False).head(numberOfItems))
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [42]:
embedded_text=embed(["Hello"])

In [43]:
similarities = cosine_similarities(embeddings.numpy(), embedded_text.numpy().T)

In [44]:
A = embeddings.numpy()
B = embedded_text.numpy().T

In [45]:
np.dot(A,B).flatten()

array([0.476579  , 0.01377824, 0.01151486, ..., 0.0474508 , 0.08687165,
       0.10028698], dtype=float32)

In [46]:
np.divide(np.dot(A,B),np.apply_along_axis(np.linalg.norm, 1, A)*np.linalg.norm(B))[:,0]

array([0.47657904, 0.01377824, 0.01151487, ..., 0.04745081, 0.08687166,
       0.10028698], dtype=float32)

In [47]:

import pickle
with open('embeddings.pickle', 'wb') as f:
    pickle.dump(embeddings, f)

In [48]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(d, f)

In [49]:
embeddings

<tf.Tensor: shape=(31103, 512), dtype=float32, numpy=
array([[ 0.00468304, -0.08392566, -0.01649407, ..., -0.03823555,
         0.00346091, -0.10069176],
       [-0.05283082, -0.00314455,  0.0539399 , ..., -0.05066366,
         0.05926255,  0.007909  ],
       [-0.04331731,  0.01895817, -0.04516518, ..., -0.00918447,
         0.02742364,  0.04344477],
       ...,
       [-0.03668367, -0.02885344,  0.04155295, ..., -0.0178898 ,
         0.09710161,  0.07535172],
       [-0.01769449, -0.03582924, -0.00357681, ...,  0.01731504,
        -0.03287543,  0.01774419],
       [-0.01537722, -0.04798326, -0.03328146, ...,  0.02343452,
         0.01298577,  0.00462453]], dtype=float32)>

In [50]:
import pickle
with open('versedict.pickle', 'wb') as f:
    pickle.dump(versedict, f)

In [51]:
d['Genesis 32:15']

'Thirty milch camels with their colts, forty kine, and ten bulls, twenty she asses, and ten foals.'

In [52]:

def cosine_similarities(A, B): #A is the big matrix containing all embeddings, B is the sentence
    return np.divide(np.dot(A,B),np.apply_along_axis(np.linalg.norm, 1, A)*np.linalg.norm(B))[:,0]

In [53]:


def generateSimilarities(text, numberOfItems):
  embedded_text = embed([text])
  similarities = cosine_similarities(embeddings.numpy(), embedded_text.numpy().T)
  data = []
  for i in range(len(embeddings)):
    data.append([versedict[i],d[versedict[i]],similarities[i]])
  return(pd.DataFrame (data, columns = ['Verse', 'Text', 'Similarity']).sort_values(by=['Similarity'], ascending=False).head(numberOfItems))

pd.set_option('max_colwidth', 500)

generateSimilarities("only son",5)


,Verse,Text,Similarity
10563,1 Chronicles 7:27,"Non his son, Jehoshuah his son.",0.454879
26063,John 1:18,"No man hath seen God at any time, the only begotten Son, which is in the bosom of the Father, he hath declared him.",0.422034
564,Genesis 22:16,"And said, By myself have I sworn, saith the LORD, for because thou hast done this thing, and hast not withheld thy son, thine only son:",0.414489
25340,Luke 9:38,"And, behold, a man of the company cried out, saying, Master, I beseech thee, look upon my son: for he is mine only child.",0.399740
27396,Acts 13:33,"God hath fulfilled the same unto us their children, in that he hath raised up Jesus again; as it is also written in the second psalm, Thou art my Son, this day have I begotten thee.",0.386814


In [54]:
generateSimilarities(d['Genesis 1:1'],5)

,Verse,Text,Similarity
1,Genesis 1:1,In the beginning God created the heaven and the earth.,1.000000
35,Genesis 2:4,"These are the generations of the heavens and of the earth when they were created, in the day that the LORD God made the earth and the heavens,",0.738225
15846,Psalms 115:15,Ye are blessed of the LORD which made heaven and earth.,0.626305
16626,Proverbs 8:23,"I was set up from everlasting, from the beginning, or ever the earth was.",0.624477
15820,Psalms 113:6,"Who humbleth himself to behold the things that are in heaven, and in the earth!",0.604734


In [55]:
import tensorflow as tf


In [56]:
from sklearn.cluster import KMeans

clusters = 10

embeddings_np = embeddings.numpy()
text = pd.DataFrame(list(d.values()))
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(embeddings_np)

In [63]:

from IPython.display import display
display.width = 50
def print_cluster(cluster_number):  
 
  locations = np.where(kmeans.labels_ == cluster_number)[0].tolist()
  text = pd.DataFrame(list(d.values()))
  display(text.iloc[locations])

In [65]:
for i in range(clusters):
  
  print_cluster(i)
  print('-'*100)




,0
20,"And God said, Let the waters bring forth abundantly the moving creature that hath life, and fowl that may fly above the earth in the open firmament of heaven."
42,"The name of the first is Pison: that is it which compasseth the whole land of Havilah, where there is gold;"
80,"So he drove out the man; and he placed at the east of the garden of Eden Cherubims, and a flaming sword which turned every way, to keep the way of the tree of life."
104,"If Cain shall be avenged sevenfold, truly Lamech seventy and sevenfold."
149,"The earth also was corrupt before God, and the earth was filled with violence."
...,...
31081,"And there shall in no wise enter into it any thing that defileth, neither whatsoever worketh abomination, or maketh a lie: but they which are written in the Lamb's book of life."
31084,And there shall be no more curse: but the throne of God and of the Lamb shall be in it; and his servants shall serve him:
31086,"And there shall be no night there; and they need no candle, neither light of the sun; for the Lord God giveth them light: and they shall reign for ever and ever."
31094,"I am Alpha and Omega, the beginning and the end, the first and the last."


----------------------------------------------------------------------------------------------------


,0
2,"And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters."
5,"And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day."
7,"And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so."
8,And God called the firmament Heaven. And the evening and the morning were the second day.
10,And God called the dry land Earth; and the gathering together of the waters called he Seas: and God saw that it was good.
...,...
31074,"The fifth, sardonyx; the sixth, sardius; the seventh, chrysolyte; the eighth, beryl; the ninth, a topaz; the tenth, a chrysoprasus; the eleventh, a jacinth; the twelfth, an amethyst."
31075,"And the twelve gates were twelve pearls: every several gate was of one pearl: and the street of the city was pure gold, as it were transparent glass."
31080,And they shall bring the glory and honour of the nations into it.
31083,"In the midst of the street of it, and on either side of the river, was there the tree of life, which bare twelve manner of fruits, and yielded her fruit every month: and the leaves of the tree were for the healing of the nations."


----------------------------------------------------------------------------------------------------


,0
69,"And the LORD God said unto the woman, What is this that thou hast done? And the woman said, The serpent beguiled me, and I did eat."
86,"And the LORD said unto Cain, Why art thou wroth? and why is thy countenance fallen?"
89,"And the LORD said unto Cain, Where is Abel thy brother? And he said, I know not: Am I my brother's keeper?"
90,"And he said, What hast thou done? the voice of thy brother's blood crieth unto me from the ground."
93,"And Cain said unto the LORD, My punishment is greater than I can bear."
...,...
31076,And I saw no temple therein: for the Lord God Almighty and the Lamb are the temple of it.
31087,"And he said unto me, These sayings are faithful and true: and the Lord God of the holy prophets sent his angel to shew unto his servants the things which must shortly be done."
31088,"Behold, I come quickly: blessed is he that keepeth the sayings of the prophecy of this book."
31090,"Then saith he unto me, See thou do it not: for I am thy fellowservant, and of thy brethren the prophets, and of them which keep the sayings of this book: worship God."


...

----------------------------------------------------------------------------------------------------


,0
1,In the beginning God created the heaven and the earth.
27,"So God created man in his own image, in the image of God created he him; male and female created he them."
35,"These are the generations of the heavens and of the earth when they were created, in the day that the LORD God made the earth and the heavens,"
40,"And out of the ground made the LORD God to grow every tree that is pleasant to the sight, and good for food; the tree of life also in the midst of the garden, and the tree of knowledge of good and evil."
49,"And the LORD God said, It is not good that the man should be alone; I will make him an help meet for him."
...,...
31077,"And the city had no need of the sun, neither of the moon, to shine in it: for the glory of God did lighten it, and the Lamb is the light thereof."
31092,"He that is unjust, let him be unjust still: and he which is filthy, let him be filthy still: and he that is righteous, let him be righteous still: and he that is holy, let him be holy still."
31095,"Blessed are they that do his commandments, that they may have right to the tree of life, and may enter in through the gates into the city."
31099,"For I testify unto every man that heareth the words of the prophecy of this book, If any man shall add unto these things, God shall add unto him the plagues that are written in this book:"


----------------------------------------------------------------------------------------------------


,0
3,"And God said, Let there be light: and there was light."
4,"And God saw the light, that it was good: and God divided the light from the darkness."
6,"And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters."
9,"And God said, Let the waters under the heaven be gathered together unto one place, and let the dry land appear: and it was so."
31,"And God saw every thing that he had made, and, behold, it was very good. And the evening and the morning were the sixth day."
...,...
31035,"And I saw an angel standing in the sun; and he cried with a loud voice, saying to all the fowls that fly in the midst of heaven, Come and gather yourselves together unto the supper of the great God;"
31054,And whosoever was not found written in the book of life was cast into the lake of fire.
31071,"And he measured the wall thereof, an hundred and forty and four cubits, according to the measure of a man, that is, of the angel."
31085,And they shall see his face; and his name shall be in their foreheads.


----------------------------------------------------------------------------------------------------
